In [ ]:
# default_exp notebook

# Notebook

> Functions to use `py2gift` from a Jupyter notebook.

In [ ]:
# export

import argparse

from IPython.core.magic import Magics, magics_class, line_magic, cell_magic, line_cell_magic

import py2gift.input_file

# Convenience functions

A class to act as a *container*. This will be useful to simulate a *Python* module.

In [ ]:
# export

class ClassesContainer:
    
    def add(cls, class_to_add):
        
        setattr(cls, class_to_add.__name__, class_to_add)

In [ ]:
classes_container = ClassesContainer()

In [ ]:
class DummyClass:
    
    n = 3

In [ ]:
classes_container.add(DummyClass)

In [ ]:
classes_container.DummyClass

In [ ]:
classes_container.DummyClass.n

# Magics

In [ ]:
# export

# The class MUST call this class decorator at creation time
@magics_class
class MyMagics(Magics):
    
    statement_key = 'statement'
    feedback_key = 'feedback'
    
    def __init__(self, shell=None,  **kwargs):
        
        super().__init__(shell=shell, **kwargs)
        
#         self.parser = argparse.ArgumentParser(description='write settings')
#         self.parser.add_argument('-n', '--name')
        
        self.location_parser = argparse.ArgumentParser(description='Specification')
        self.location_parser.add_argument('settings', help='Settings object')
        self.location_parser.add_argument('-c', '--class', default=None, help='class')
        
        # the name of a category can contain spaces; notice that this will yield a *list* rather than a string
        self.location_parser.add_argument('-C', '--category', default=None, nargs='+', help='category')
    
#     def process(self, line, cell, variable):
    def process(self, line, cell, key):
        
        line_arguments = self.location_parser.parse_args(line.split())
        
        print(f'{line_arguments.category=}')
        
        if line_arguments.category:
            
            # this is a list
            category = ' '.join(line_arguments.category).split(',')
            
            if len(category) == 1:
                
                category = category[0]
        
        else:
            
            category = None
            
#             line_arguments.category = ' '.join(line_arguments.category)
        
        self.shell.user_ns[line_arguments.settings].store['categories'][-1]['classes'][-1][key] = cell
        
        print(f'{line_arguments=}')
        print(f'processed category={category}')

#     @line_magic
#     def lmagic(self, line):
        
#         "my line magic"
#         print("Full access to the main IPython object:", self.shell)
#         print("Variables in the user namespace:", list(self.shell.user_ns.keys()))
        
#         line_arguments = self.parser.parse_args(line.split())
#         print(line_arguments)
        
#         return line

    @cell_magic
    def statement(self, line, cell):

        self.process(line, cell, self.statement_key)
        
        return f'statement recorded'
    
    @cell_magic
    def feedback(self, line, cell):
        
        self.process(line, cell, self.feedback_key)
        
        return f'feedback recorded'

#     @line_cell_magic
#     def lcmagic(self, line, cell=None):
        
#         "Magic that works both as %lcmagic and as %%lcmagic"
#         if cell is None:
#             print("Called as line magic")
#             return line
#         else:
#             print("Called as cell magic")
#             return line, cell

## Parser

The class can be instantiated directly,

In [ ]:
m = MyMagics()

to test the parser,

In [ ]:
m.location_parser.parse_args('settings -c myclass -C mycategory'.split())

When no category nor class are passed,

In [ ]:
m.location_parser.parse_args('settings'.split())

In [ ]:
line = m.location_parser.parse_args('settings -c myc -C cat 1,sub cat 2'.split())
' '.join(line.category).split(',')

In [ ]:
line = m.location_parser.parse_args('settings -c myc -C cat 1'.split())
' '.join(line.category).split(',')

## Actual use

*Magics* must be somehow registered

In [ ]:
# export

get_ipython().register_magics(MyMagics)

In [ ]:
# %lmagic -n foo

In [ ]:
settings = py2gift.input_file.Settings('out.yaml', 'pics')
category_name = settings.add_category('Category 1')
settings.add_or_update_class(category_name=category_name, class_name='question name', question_base_name='base name', n_instances=2)

In [ ]:
settings.to_dict()

In [ ]:
%%statement settings
Consider

bla bla

In [ ]:
settings

In [ ]:
cls = 'ClassA'
category = 'Category 1'

In [ ]:
%%statement settings -c {cls} -C {category}
more blah

In [ ]:
%%statement settings -C foo foo -c cls 
more

In [ ]:
%%feedback  settings -c ClassB -C wap wap,oh
We must...

In [ ]:
settings

In [ ]:
# hide

import nbdev.export
nbdev.export.notebook2script('60_notebook.ipynb')